y(label)와 파생 변수 생성

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

batter = pd.read_csv('./data/batter.csv', encoding='utf-8-sig')
pitcher = pd.read_csv('./data/pitcher.csv', encoding='utf-8-sig')

In [2]:
batter.head()

,이름,연도,소속,WAR,G,타석,타수,득점,안타,2타,...,삼진,병살,희타,희비,타율,출루,장타,OPS,wOBA,wRC+
0,백인천,1982,MBC,5.88,72,298,250,55,103,23,...,17,5,0,3,0.412,0.497,0.740,1.237,0.528,227.0
1,윤동균,1982,OB,4.13,77,334,284,54,97,20,...,27,3,0,4,0.342,0.428,0.532,0.960,0.430,164.6
2,이만수,1982,삼성,4.12,78,322,270,46,78,16,...,27,13,0,1,0.289,0.399,0.500,0.899,0.405,147.6
3,김봉연,1982,해태,3.68,74,304,269,55,89,14,...,16,8,0,1,0.331,0.405,0.636,1.040,0.453,180.5
4,김우열,1982,OB,3.46,62,255,210,43,65,8,...,25,0,0,1,0.310,0.428,0.533,0.961,0.426,161.9


In [3]:
pitcher.head()

,이름,연도,소속,WAR,출장,완투,완봉,선발,승,패,...,고4,사구,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+
0,박철순,1982,OB,9.54,36,15,2,19,24,4,...,1,6,108,0,4,1.84,2.82,0.97,211.2,137.8
1,황규봉,1982,삼성,8.62,47,8,2,16,15,11,...,3,6,99,1,4,2.47,2.67,1.00,160.6,148.5
2,노상수,1982,롯데,7.05,44,12,2,20,14,19,...,6,10,141,2,4,2.94,3.03,1.18,134.9,131.3
3,하기룡,1982,MBC,6.78,43,7,4,16,13,10,...,4,13,107,2,6,2.30,2.95,1.11,172.3,134.4
4,권영호,1982,삼성,5.12,32,6,3,21,15,5,...,0,7,72,1,1,2.37,3.91,1.07,167.5,101.3


## 타이틀홀더 변수 생성

In [4]:
b_title = pd.read_csv('./data/batter_titleholder.csv', encoding='cp949')
b_title.head()

,연도,타율,안타,홈런,타점,득점,도루,장타율,출루율
0,1982,백인천,백인천,김봉연,김성한,"김봉연, 백인천",김일권,백인천,백인천
1,1983,장효조,"박종훈, 장효조",이만수,이만수,이해창,김일권,장효조,장효조
2,1984,이만수,홍문종,이만수,이만수,"이해창, 홍문종",김일권,이만수,장효조
3,1985,장효조,김성한,"김성한, 이만수",이만수,이순철,김재박,김성한,장효조
4,1986,장효조,이광은,김봉연,김봉연,김재박,서정환,김봉연,장효조


In [21]:
b_title_data = []

for i in range(len(b_title)):
    for j in range(1, 9):
        year = b_title.iloc[i, 0]
        names = map(lambda x:x.strip(), b_title.iloc[i, j].split(','))
        for name in names:
            b_title_data.append([year, name, 1])

b_title_df = pd.DataFrame(b_title_data, columns=['연도', '이름', '홀더'])
b_title_df = b_title_df.groupby(['연도', '이름']).sum().reset_index()

In [22]:
b_title_df.head()

,연도,이름,홀더
0,1982,김봉연,2
1,1982,김성한,1
2,1982,김일권,1
3,1982,백인천,5
4,1983,김일권,1


In [23]:
p_title = pd.read_csv('./data/pitcher_titleholder.csv', encoding='cp949')
p_title.head()

,연도,다승,평균자책점,탈삼진,세이브,홀드,승률
0,1982,박철순,박철순,노상수,황규봉,NaN,박철순
1,1983,장명부,하기룡,장명부,황태환,NaN,이길환
2,1984,최동원,장호연,최동원,윤석환,NaN,황규봉
3,1985,"김시진, 김일융",선동열,김지신,권영호,NaN,"김시진, 윤석환"
4,1986,선동열,선동열,선동열,김용수,NaN,최일언


In [24]:
p_title_data = []

for i in range(len(p_title)):
    for j in range(1, 7):
        year = p_title.iloc[i, 0]
        if pd.isna(p_title.iloc[i, j]):
            continue
        names = map(lambda x:x.strip(), p_title.iloc[i, j].split(','))
        for name in names:
            p_title_data.append([year, name, 1])

p_title_df = pd.DataFrame(p_title_data, columns=['연도', '이름', '홀더'])
p_title_df = p_title_df.groupby(['연도', '이름']).sum().reset_index()

In [26]:
p_title_df.head()

,연도,이름,홀더
0,1982,노상수,1
1,1982,박철순,3
2,1982,황규봉,1
3,1983,이길환,1
4,1983,장명부,2


In [27]:
batter = pd.merge(batter, b_title_df, how='left', on=['연도', '이름'])
batter.fillna(0, inplace=True)
batter.head()

,이름,연도,소속,WAR,G,타석,타수,득점,안타,2타,...,병살,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,홀더
0,백인천,1982,MBC,5.88,72,298,250,55,103,23,...,5,0,3,0.412,0.497,0.740,1.237,0.528,227.0,5.0
1,윤동균,1982,OB,4.13,77,334,284,54,97,20,...,3,0,4,0.342,0.428,0.532,0.960,0.430,164.6,0.0
2,이만수,1982,삼성,4.12,78,322,270,46,78,16,...,13,0,1,0.289,0.399,0.500,0.899,0.405,147.6,0.0
3,김봉연,1982,해태,3.68,74,304,269,55,89,14,...,8,0,1,0.331,0.405,0.636,1.040,0.453,180.5,2.0
4,김우열,1982,OB,3.46,62,255,210,43,65,8,...,0,0,1,0.310,0.428,0.533,0.961,0.426,161.9,0.0


In [28]:
pitcher = pd.merge(pitcher, p_title_df, how='left', on=['연도', '이름'])
pitcher.fillna(0, inplace=True)
pitcher.head()

,이름,연도,소속,WAR,출장,완투,완봉,선발,승,패,...,사구,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,홀더
0,박철순,1982,OB,9.54,36,15,2,19,24,4,...,6,108,0,4,1.84,2.82,0.97,211.2,137.8,3.0
1,황규봉,1982,삼성,8.62,47,8,2,16,15,11,...,6,99,1,4,2.47,2.67,1.00,160.6,148.5,1.0
2,노상수,1982,롯데,7.05,44,12,2,20,14,19,...,10,141,2,4,2.94,3.03,1.18,134.9,131.3,1.0
3,하기룡,1982,MBC,6.78,43,7,4,16,13,10,...,13,107,2,6,2.30,2.95,1.11,172.3,134.4,0.0
4,권영호,1982,삼성,5.12,32,6,3,21,15,5,...,7,72,1,1,2.37,3.91,1.07,167.5,101.3,0.0


## 우승팀 소속 변수 생성

In [29]:
ks = pd.read_csv('./data/ks.csv', encoding='cp949')
ks.head()

,연도,소속
0,1982,OB
1,1983,해태
2,1984,롯데
3,1985,삼성
4,1986,해태


In [30]:
ks['우승'] = 1

In [31]:
batter = pd.merge(batter, ks, on=['연도', '소속'], how='left')
batter.fillna(0, inplace=True)
batter.head()

,이름,연도,소속,WAR,G,타석,타수,득점,안타,2타,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,홀더,우승
0,백인천,1982,MBC,5.88,72,298,250,55,103,23,...,0,3,0.412,0.497,0.740,1.237,0.528,227.0,5.0,0.0
1,윤동균,1982,OB,4.13,77,334,284,54,97,20,...,0,4,0.342,0.428,0.532,0.960,0.430,164.6,0.0,1.0
2,이만수,1982,삼성,4.12,78,322,270,46,78,16,...,0,1,0.289,0.399,0.500,0.899,0.405,147.6,0.0,0.0
3,김봉연,1982,해태,3.68,74,304,269,55,89,14,...,0,1,0.331,0.405,0.636,1.040,0.453,180.5,2.0,0.0
4,김우열,1982,OB,3.46,62,255,210,43,65,8,...,0,1,0.310,0.428,0.533,0.961,0.426,161.9,0.0,1.0


In [32]:
pitcher = pd.merge(pitcher, ks, on=['연도', '소속'], how='left')
pitcher.fillna(0, inplace=True)
pitcher.head()

,이름,연도,소속,WAR,출장,완투,완봉,선발,승,패,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,홀더,우승
0,박철순,1982,OB,9.54,36,15,2,19,24,4,...,108,0,4,1.84,2.82,0.97,211.2,137.8,3.0,1.0
1,황규봉,1982,삼성,8.62,47,8,2,16,15,11,...,99,1,4,2.47,2.67,1.00,160.6,148.5,1.0,0.0
2,노상수,1982,롯데,7.05,44,12,2,20,14,19,...,141,2,4,2.94,3.03,1.18,134.9,131.3,1.0,0.0
3,하기룡,1982,MBC,6.78,43,7,4,16,13,10,...,107,2,6,2.30,2.95,1.11,172.3,134.4,0.0,0.0
4,권영호,1982,삼성,5.12,32,6,3,21,15,5,...,72,1,1,2.37,3.91,1.07,167.5,101.3,0.0,0.0


y라벨 생성

In [33]:
y_base = pd.read_csv('./data/MVP.csv', encoding='cp949')
y_base.head()

,연도,MVP,신인상
0,2021,미란다,이의리
1,2020,로하스,소형준
2,2019,린드블럼,정우영
3,2018,김재환,강백호
4,2017,양현종,이정후


In [34]:
y_base.rename(columns={'MVP':'이름'}, inplace=True)
y_base.head()

,연도,이름,신인상
0,2021,미란다,이의리
1,2020,로하스,소형준
2,2019,린드블럼,정우영
3,2018,김재환,강백호
4,2017,양현종,이정후


In [35]:
y_base['y'] = 1
y_base.drop(columns=['신인상'], inplace=True)
y_base.head()

,연도,이름,y
0,2021,미란다,1
1,2020,로하스,1
2,2019,린드블럼,1
3,2018,김재환,1
4,2017,양현종,1


In [36]:
batter = pd.merge(batter, y_base, on=['연도', '이름'], how='left')
pitcher = pd.merge(pitcher, y_base, on=['연도', '이름'], how='left')

batter.fillna(0, inplace=True)
pitcher.fillna(0, inplace=True)

In [37]:
batter.head()

,이름,연도,소속,WAR,G,타석,타수,득점,안타,2타,...,희비,타율,출루,장타,OPS,wOBA,wRC+,홀더,우승,y
0,백인천,1982,MBC,5.88,72,298,250,55,103,23,...,3,0.412,0.497,0.740,1.237,0.528,227.0,5.0,0.0,0.0
1,윤동균,1982,OB,4.13,77,334,284,54,97,20,...,4,0.342,0.428,0.532,0.960,0.430,164.6,0.0,1.0,0.0
2,이만수,1982,삼성,4.12,78,322,270,46,78,16,...,1,0.289,0.399,0.500,0.899,0.405,147.6,0.0,0.0,0.0
3,김봉연,1982,해태,3.68,74,304,269,55,89,14,...,1,0.331,0.405,0.636,1.040,0.453,180.5,2.0,0.0,0.0
4,김우열,1982,OB,3.46,62,255,210,43,65,8,...,1,0.310,0.428,0.533,0.961,0.426,161.9,0.0,1.0,0.0


In [38]:
pitcher.head()

,이름,연도,소속,WAR,출장,완투,완봉,선발,승,패,...,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,홀더,우승,y
0,박철순,1982,OB,9.54,36,15,2,19,24,4,...,0,4,1.84,2.82,0.97,211.2,137.8,3.0,1.0,1.0
1,황규봉,1982,삼성,8.62,47,8,2,16,15,11,...,1,4,2.47,2.67,1.00,160.6,148.5,1.0,0.0,0.0
2,노상수,1982,롯데,7.05,44,12,2,20,14,19,...,2,4,2.94,3.03,1.18,134.9,131.3,1.0,0.0,0.0
3,하기룡,1982,MBC,6.78,43,7,4,16,13,10,...,2,6,2.30,2.95,1.11,172.3,134.4,0.0,0.0,0.0
4,권영호,1982,삼성,5.12,32,6,3,21,15,5,...,1,1,2.37,3.91,1.07,167.5,101.3,0.0,0.0,0.0


In [40]:
batter.to_csv('./data/batter_dataset.csv', encoding='utf-8-sig', index=False)
pitcher.to_csv('./data/pitcher_dataset.csv', encoding='utf-8-sig', index=False)